# LangchainNotionToolkit

This notebook provides a quick overview for getting started with **LangchainNotionToolkit**.
For detailed documentation of all LangchainNotionToolkit features and configurations head to the
[API reference](https://python.langchain.com/v0.2/api_reference/community/toolkits/langchain_community.toolkits.langchain_notion.LangchainNotionToolkit.html).

You may also want to check the official [Notion API docs](https://developers.notion.com/docs)
for details on objects like pages, databases, and properties.

---

## Overview

### Integration details

| Class | Package | Serializable | JS support | Package latest |
| :--- | :--- | :---: | :---: | :---: |
| `LangchainNotionToolkit` | [`langchain-notion`](https://pypi.org/project/langchain-notion/) | ❌ | ❌ | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-notion?style=flat-square&label=%20) |

### Tool features

The `LangchainNotionToolkit` bundles multiple tools for interacting with Notion through agents:

| Tool | Description | Input Schema |
| :--- | :--- | :--- |
| Search Pages | Search for pages in Notion using keywords. | `{"query": str}` _or_ a plain string |
| Get Page | Retrieve a page’s title, URL, and properties by ID. | `{"page_id": str}` |
| Create Page | Create a new page in a database or under another page. | `{"parent": {...}, "properties": {...}}` |
| Update Page | Update properties of an existing page. | `{"page_id": str, "properties": {...}}` |

---

## Setup

Install dependencies:

```bash
pip install langchain-notion
```

In [ ]:
%pip install --quiet -U langchain-notion

### Credentials

This integration requires a **Notion API key** (integration token).

1. Create an integration at the [Notion Developer Portal](https://www.notion.so/my-integrations) and copy the token (`secret_...`).
2. Share your target pages or databases with this integration (Notion → **Share** → invite your integration).
3. Provide credentials to this notebook via environment variables or directly to the wrapper.

**Environment variables (recommended):**

```bash
export NOTION_API_KEY="secret_abc..."
export NOTION_DATABASE_ID="your_database_id"   # optional (for create)
```

In [ ]:
import getpass, os

if not os.environ.get("NOTION_API_KEY"):
    os.environ["NOTION_API_KEY"] = getpass.getpass("Enter your Notion API key (secret_...):\n")

It's also helpful (but not required) to set up [LangSmith](https://smith.langchain.com/) for observability:

In [ ]:
# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API key:\n")

## Instantiation

Instantiate the toolkit with a `NotionWrapper`:

In [ ]:
from langchain_notion.notion_wrapper import NotionWrapper
from langchain_notion.toolkits import LangchainNotionToolkit

api = NotionWrapper(api_key=os.environ["NOTION_API_KEY"])  # will also read NOTION_DATABASE_ID if set
toolkit = LangchainNotionToolkit.from_notion_wrapper(api, include_write_tools=True)
tools = toolkit.get_tools()
tools

## Invocation

### Invoke directly with args

Each tool can be retrieved from the toolkit and invoked directly.

**Search for pages:**

In [ ]:
search = next(t for t in tools if t.name == "Search Pages")
print(search.invoke("roadmap"))  # plain string is supported
# print(search.invoke({"query": "roadmap"}))  # dict form also supported

**Get a page:**

In [ ]:
get_page = next(t for t in tools if t.name == "Get Page")
print(get_page.invoke({"page_id": "YOUR_PAGE_ID"}))  # replace with a real page ID shared with the integration

**Create a page:**

In [ ]:
create_page = next(t for t in tools if t.name == "Create Page")
print(create_page.invoke({
    "parent": {"database_id": os.environ.get("NOTION_DATABASE_ID", "YOUR_DATABASE_ID")},
    "properties": {
        "title": {"title": [{"type": "text", "text": {"content": "Demo Page"}}]}
    }
}))

**Update a page:**

In [ ]:
update_page = next(t for t in tools if t.name == "Update Page")
print(update_page.invoke({
    "page_id": "YOUR_PAGE_ID",
    "properties": {
        "title": {"title": [{"type": "text", "text": {"content": "Updated Title"}}]}
    }
}))

## Chaining

You can also bind these tools to a chat model that supports tool calling.
Below is a minimal example using `init_chat_model` (you will need valid model credentials):

In [ ]:
# !pip install -qU langchain langchain-openai
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

llm = init_chat_model(model="gpt-4o", model_provider="openai")
llm_with_tools = llm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful Notion assistant."),
    ("human", "Search for pages about the project roadmap")
])

chain = prompt | llm_with_tools
result = chain.invoke({})
print(result)

## API reference

For detailed documentation of all LangchainNotionToolkit features and configurations, see the
[API reference](https://python.langchain.com/v0.2/api_reference/community/toolkits/langchain_community.toolkits.langchain_notion.LangchainNotionToolkit.html).